# Functions for grabbing data from WRC API

This package contains a range of functions for grabbing and parsing live timing results data from the WRC website via a simple JSON API that is used to generate the official WRC live timing results web pages.


TO DO - consider a scraper class with a requests session embedded in it.

In [64]:
%load_ext autoreload
%autoreload 2

%load_ext pycodestyle_magic
%flake8_on --ignore D100

In [65]:
import requests
import warnings
import json
import pandas as pd
from pandas.io.json import json_normalize

In [66]:
# Cache results in text notebook
import requests_cache
requests_cache.install_cache('wrc_cache',
                             backend='sqlite',
                             expire_after=30000)

In [68]:
# TO DO
# There is also an: activeSeasonId":19
# Is there something we can get there?

In [69]:
from WRCUtils2020 import _isnull, _notnull, _checkattr, _jsInt, listify

In [71]:
# Is this URL constant or picked up relative to each rally?
URL = 'https://www.wrc.com/ajax.php?contelPageId=176146'

In [77]:
def _getresponse(_url, args, ss={'conn': None}, secondtry=False):
    """Get response from a post request."""
    r = None
    if ss['conn'] is None or secondtry:
        try:
            ss['conn'] = requests.Session()
            ss['conn'].get('https://www.wrc.com')
        except:
            return None
        
    try:
        r = ss['conn'].post(_url, data = json.dumps(args))
    except: #requests.exceptions.ConnectionError:
        if not secondtry:
            #If there's an error, try once again
            try:
                _getresponse(_url, args, secondtry = True)
            except:
                return None
        else:
            return None
            
    return r

1:33: B006 Do not use mutable data structures for argument defaults.  They are created during function definition time. All calls to the function reuse this one instance of that data structure, persisting changes between them.
8:9: E722 do not use bare 'except'


Error in callback <bound method VarWatcher.auto_run_flake8 of <pycodestyle_magic.VarWatcher object at 0x118af1d50>> (for post_run_cell):


ValueError: invalid literal for int() with base 10: '`, it also catches unexpected events like memory errors, interrupts, system exit, and so on.  Prefer `except Exception'

In [89]:
def _get_and_handle_response(_url, args, func, nargs=1,
                             raw=False, renamecols=None,
                             extracols=None, dropcols=None):
    """
    Make request to WRC API.

    Return a raw string or parse the response
    with a provided parser function.
    """
    def _add_cols(response, extracols):
        """Add extra columns to each dataframe."""
        if _isnull(response):
            return

        dupes = set(response.columns).intersection(extracols.keys())

        if dupes:
            warnings.warn(f"Trying to add pre-existing cols: {', '.join(dupes)}")
        for k in extracols:
            response[k] = extracols[k]

    r = _getresponse(_url, args)
    if raw or not callable(func):
        return r.text

    # Make sure we return the desired number of None items
    # in a tuple as a null response
    if not r or r is None or not r.text or r.text == 'null':
        return tuple([None for i in range(nargs)])

    response = func(r)

    if renamecols is None:
        renamecols = {}
    if extracols is None:
        extracols = {}

    # The dataframe type check can help if we have the wrong number of args
    # Could display a warning that the nargs is incorrect if so
    if nargs == 1 or isinstance(response, pd.DataFrame):
        if renamecols:
            response.rename(columns=renamecols, inplace=True)
        if extracols:
            _add_cols(response, extracols)
        if dropcols:
            response.drop(columns=dropcols, inplace=True, errors='ignore')
    else:
        for i in range(nargs):
            if renamecols:
                _cols = response[i].columns
                _cols = set(_cols).intersection(renamecols.keys())
                # Couldn't we just errors='ignore?
                response[i].rename(columns={k: renamecols[k] for k in _cols},
                                   inplace=True, errors='ignore')
            if extracols:
                _add_cols(response[i], extracols)
            if dropcols:
                response[i].drop(columns=dropcols,
                                 inplace=True, errors='ignore')

    return response

1:1: E302 expected 2 blank lines, found 0
18:80: E501 line too long (81 > 79 characters)


In [90]:
ACTIVE_RALLY_URL = 'https://www.wrc.com/ajax.php?contelPageId=171091'

In [96]:
def _parseActiveRally(r):
    """Parse active rally response."""
    event = json_normalize(r.json()).drop(columns='eventDays')
    days = json_normalize(r.json(),
                          'eventDays').drop(columns='spottChannel.assets')
    channels = json_normalize(r.json(),
                              ['eventDays', 'spottChannel', 'assets'])
    return (event, days, channels)


def getActiveRally(_url=None, raw=False, func=_parseActiveRally):
    """Get active rally details."""
    if not _url:
        _url = ACTIVE_RALLY_URL
    args = {"command": "getActiveRally", "context": None}

    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw,
                                    dropcols='winner.driverImageFormats')


1:6: N802 function name '_parseActiveRally' should be lowercase
11:6: N802 function name 'getActiveRally' should be lowercase


In [97]:
event, days, channels = getActiveRally()  # Also works with passing URL
display(event.head())
display(days.head())
display(channels.head())
display(event.columns)

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.nation.alpha2,winner.nation.alpha3,winner.nation.ioc,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.externalId,winner.page
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,BE,BEL,BEL,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,762,None


,id,eventDay,spottChannel.id,spottChannel.displayName
0,334,2020-01-23,2,WRC Rallye Monte Carlo
1,341,2020-01-24,2,WRC Rallye Monte Carlo
2,344,2020-01-25,2,WRC Rallye Monte Carlo
3,355,2020-01-26,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.400x225,content.image.800x450,content.image.thumbnail,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
1,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,241,2020-01-23T20:30:00+00:00,1579811400,2020-01-23T21:15:00+00:00,1579814100,0,Break,None,https://ott.wrc.com/image/480/270/5e25b3a530fc...,https://ott.wrc.com/image/thumbnail/5e25b3a530...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.exter

1:41: E261 at least two spaces before inline comment
1:42: E262 inline comment should start with '# '


In [11]:
# Raw https://webappsdata.wrc.com/srv API?
# Need to create separate package to query that API
# Season info
# _url = 'https://webappsdata.wrc.com/srv/wrc/json/api/wrcsrv/byType?t=%22Season%22&maxdepth=1' 
# r = s.get(_url)
# json_normalize(r.json())

In [98]:
CURRENT_SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=181782'

In [102]:
def _parseActiveSeasonEvents(r):
    """Parse current season events response."""
    current_season_events = json_normalize(r.json(),
                                           ['rallyEvents', 'items'],
                                           meta='seasonYear').drop(columns='eventDays')
    eventdays = json_normalize(r.json(),
                               ['rallyEvents', 'items',
                                'eventDays']).drop(columns='spottChannel.assets')
    eventchannel = json_normalize(r.json(),
                                  ['rallyEvents', 'items', 'eventDays',
                                   'spottChannel', 'assets'])
    return (current_season_events, eventdays, eventchannel)


# TO DO - can we get events for other seasons?
def getActiveSeasonEvents(raw=False, func=_parseActiveSeasonEvents):
    """Get events for current season."""
    _url = CURRENT_SEASON_URL
    # There seems to be a second UTL giving same data?
    # _url='https://www.wrc.com/ajax.php?contelPageId=183400'
    args = {"command": "getActiveSeason", "context": None}

    return _get_and_handle_response(_url, args, func, nargs=3, raw=raw)
    # dropcols='winner.driverImageFormats')


1:6: N802 function name '_parseActiveSeasonEvents' should be lowercase
5:80: E501 line too long (87 > 79 characters)
8:80: E501 line too long (81 > 79 characters)
16:6: N802 function name 'getActiveSeasonEvents' should be lowercase


In [103]:
current_season_events, eventdays, eventchannel = getActiveSeasonEvents()
display(current_season_events.head())
display(eventdays.head())
display(eventchannel.head())
display(current_season_events.columns)
eventchannel.columns

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.driverImageFormats,winner.externalId,winner.page,winner,seasonYear
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,"[{'id': 6, 'title': 'Format 16:9', 'imageForma...",762.0,NaN,NaN,2020
1,102,Rally Sweden,154,125,2,False,True,True,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
2,107,Rally Guanajuato Mexico,155,126,-6,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
3,114,Rally Argentina,156,127,-3,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
4,116,Rally de Portugal,157,128,1,False,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


,id,eventDay,spottChannel.id,spottChannel.displayName
0,334,2020-01-23,2,WRC Rallye Monte Carlo
1,341,2020-01-24,2,WRC Rallye Monte Carlo
2,344,2020-01-25,2,WRC Rallye Monte Carlo
3,355,2020-01-26,2,WRC Rallye Monte Carlo
4,363,2020-02-13,2,WRC Rallye Monte Carlo


,id,start,startUnix,end,endUnix,duration,alternative.title,alternative.description,alternative.image.480x270,alternative.image.thumbnail,...,content.image.400x225,content.image.800x450,content.image.thumbnail,content.dateTime.date,content.dateTime.timezone_type,content.dateTime.timezone,content.payment.id,content.payment.name,content.status.id,content.status.name
0,64,2020-01-23T18:00:00+00:00,1579802400,2020-01-23T18:30:00+00:00,1579804200,0,Good Evening Rally Fans - Service Gap,None,https://ott.wrc.com/image/480/270/5e2618150992...,https://ott.wrc.com/image/thumbnail/5e26181509...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
1,65,2020-01-23T18:30:00+00:00,1579804200,2020-01-23T19:15:00+00:00,1579806900,0,eSPORTS (TV Live),None,https://ott.wrc.com/image/480/270/5e1e100fce9e...,https://ott.wrc.com/image/thumbnail/5e1e100fce...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
2,239,2020-01-23T19:15:00+00:00,1579806900,2020-01-23T19:30:00+00:00,1579807800,0,Break,None,https://ott.wrc.com/image/480/270/5e25b39e1be1...,https://ott.wrc.com/image/thumbnail/5e25b39e1b...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
3,68,2020-01-23T19:30:00+00:00,1579807800,2020-01-23T20:30:00+00:00,1579811400,0,SS1 Malijai - Puimichel (TV LIVE),None,https://ott.wrc.com/image/480/270/5e1dfc31d968...,https://ott.wrc.com/image/thumbnail/5e1dfc31d9...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live
4,241,2020-01-23T20:30:00+00:00,1579811400,2020-01-23T21:15:00+00:00,1579814100,0,Break,None,https://ott.wrc.com/image/480/270/5e25b3a530fc...,https://ott.wrc.com/image/thumbnail/5e25b3a530...,...,https://ott.wrc.com/image/400/225/placeholder....,https://ott.wrc.com/image/800/450/placeholder....,https://ott.wrc.com/image/thumbnail/placeholde...,2020-01-23 18:00:00.000000,1,+00:00,3,Pay,3,Live


Index(['id', 'name', 'externalIdRally', 'externalIdEvent', 'timezone',
       'active', 'countdown', 'jwrc', 'images.format16x9.320x180',
       'images.format16x9.160x90', 'images.format16x9.path', 'season.id',
       'season.year', 'season.externalId', 'season.active', 'rally.id',
       'rally.name', 'rally.nation.id', 'rally.nation.name',
       'rally.nation.isoNumCode', 'rally.nation.alpha2', 'rally.nation.alpha3',
       'rally.nation.ioc', 'status.id', 'status.name', 'pageInfo.id',
       'pageInfo.title', 'pageInfo.feTitle', 'pageInfo.url', 'pageResult.id',
       'pageResult.title', 'pageResult.feTitle', 'pageResult.url', 'winner.id',
       'winner.firstName', 'winner.middleName', 'winner.lastName',
       'winner.nation.id', 'winner.nation.name', 'winner.nation.isoNumCode',
       'winner.nation.alpha2', 'winner.nation.alpha3', 'winner.nation.ioc',
       'winner.birthDate', 'winner.birthPlace', 'winner.debutDate',
       'winner.debutPlace', 'winner.website', 'winner.drive

Index(['id', 'start', 'startUnix', 'end', 'endUnix', 'duration',
       'alternative.title', 'alternative.description',
       'alternative.image.480x270', 'alternative.image.thumbnail',
       'alternative.image.original', 'content.id', 'content.title',
       'content.description', 'content.image.original',
       'content.image.1920x1080', 'content.image.400x225',
       'content.image.800x450', 'content.image.thumbnail',
       'content.dateTime.date', 'content.dateTime.timezone_type',
       'content.dateTime.timezone', 'content.payment.id',
       'content.payment.name', 'content.status.id', 'content.status.name'],
      dtype='object')

## getItinerary

In [107]:
# This seems to work with sdbRallyId=None, returning active rally?

def _parseItinerary(r):
    """Parse itinerary response."""
    itinerary = json_normalize(r.json()).drop(columns='itineraryLegs')
    legs = json_normalize(r.json(), 'itineraryLegs')
    if not legs.empty:
        legs = legs.drop(columns='itinerarySections')
        sections = json_normalize(r.json(),
                                  ['itineraryLegs', 'itinerarySections']).drop(columns=['controls', 'stages'])
        controls = json_normalize(r.json(),
                                  ['itineraryLegs', 'itinerarySections', 'controls'])
        stages = json_normalize(r.json(),
                                ['itineraryLegs', 'itinerarySections', 'stages'])
    else:
        legs = sections = controls = stages = None
    return (itinerary, legs, sections, controls, stages)


def getItinerary(sdbRallyId=None, raw=False, func=_parseItinerary):
    """Get itinerary details for specified rally."""
    if not sdbRallyId:
        event, days, channels = getActiveRally()
        sdbRallyId = int(event.loc[0, 'id'])

    args = {"command": "getItinerary",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}
    if sdbRallyId:
        extracols = {'rallyid': sdbRallyId}
    else:
        extracols = {}
    # TO DO - could we annotate with a looked up rally id?
    #Presumably from eg getActiveRally()? Or more generally ActiveSeasonEvents
    return _get_and_handle_response(URL, args, func, nargs=5,
                                    raw=raw, extracols = extracols)


3:1: E302 expected 2 blank lines, found 1
3:6: N802 function name '_parseItinerary' should be lowercase
10:73: E202 whitespace before ')'
10:80: E501 line too long (111 > 79 characters)
12:80: E501 line too long (85 > 79 characters)
14:80: E501 line too long (81 > 79 characters)
20:6: N802 function name 'getItinerary' should be lowercase
20:19: N803 argument name 'sdbRallyId' should be lowercase
24:10: N806 variable 'sdbRallyId' in function should be lowercase
33:5: E265 block comment should start with '# '
35:55: E251 unexpected spaces around keyword / parameter equals
35:57: E251 unexpected spaces around keyword / parameter equals


In [108]:
sdbRallyId = 100
itinerary, legs, sections, controls, stages = getItinerary(sdbRallyId)
display(itinerary.head())
display(legs.head())
display(sections.head())
display(controls.head())
display(stages.head())

,itineraryId,eventId,name,priority,rallyid
0,144,78,Itinerary,1,100


,itineraryLegId,itineraryId,startListId,name,legDate,order,status,rallyid
0,155,144,255,Thursday 24th January,2019-01-24,1,Completed,100
1,154,144,259,Friday 25th January,2019-01-25,2,Completed,100
2,153,144,260,Saturday 26th January,2019-01-26,3,Completed,100
3,152,144,262,Sunday 27th January,2019-01-27,4,Running,100


,itinerarySectionId,itineraryLegId,order,name,rallyid
0,362,155,1,Section 1,100
1,361,154,2,Section 2,100
2,360,154,3,Section 3,100
3,359,153,4,Section 4,100
4,358,153,5,Section 5,100


,controlId,eventId,stageId,type,code,location,timingPrecision,distance,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked,rallyid
0,3667,78,880.0,TimeControl,TC0,GAP (Place Desmichels),Minute,NaN,None,NaN,2019-01-24T17:50:00,2019-01-24T18:50:00+01:00,Completed,All,NoRounding,False,100
1,3720,78,881.0,TimeControl,TC1,LA BREOLE,Minute,32.52,00:45:00,2700000.0,2019-01-24T18:35:00,2019-01-24T19:35:00+01:00,Completed,All,NoRounding,False,100
2,3721,78,881.0,StageStart,SS1,LA BREOLE - SELONNET,Minute,20.76,00:03:00,180000.0,2019-01-24T18:38:00,2019-01-24T19:38:00+01:00,Completed,None,RoundToClosestMinute,False,100
3,3722,78,881.0,FlyingFinish,SF1,LA BREOLE - SELONNET,Tenth,NaN,None,NaN,None,None,Completed,None,NoRounding,False,100
4,3723,78,885.0,TimeControl,TC2,AVANÇON,Minute,32.04,01:00:00,3600000.0,2019-01-24T19:38:00,2019-01-24T20:38:00+01:00,Completed,All,NoRounding,False,100


,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code,rallyid
0,881,78,1,LA BREOLE - SELONNET,20.76,Completed,SpecialStage,Tenth,False,SS1,100
1,885,78,2,AVANÇON - NOTRE DAME DU LAUS,20.59,Completed,SpecialStage,Tenth,False,SS2,100
2,894,78,3,VALDROME - SIGOTTIER 1,20.04,Cancelled,SpecialStage,Tenth,False,SS3,100
3,886,78,4,ROUSSIEUX - LABOREL 1,24.05,Interrupted,SpecialStage,Tenth,False,SS4,100
4,888,78,5,CURBANS - PIEGUT 1,18.47,Interrupted,SpecialStage,Tenth,False,SS5,100


1:2: N816 variable 'sdbRallyId' in global scope should not be mixedCase


In [110]:
def _parseStartlist(r):
    """Parse raw startlist response."""
    startList = json_normalize(r.json()).drop(columns='startListItems')
    startListItems = json_normalize(r.json(), 'startListItems')

    return (startList, startListItems)


# TO DO - so can get a startlist for a not active leg?

def getActiveLegStartlist(startListId, raw=False, func=_parseStartlist):
    """Get a startlist for the active itinerary leg."""
    args = {'command': 'getStartlist',
            'context': {'activeItineraryLeg': {'startListId': startListId}}}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)


def getStartlist(startListId, typ='activeleg', raw=False):
    """Get a generic startlist."""
    if typ == 'activeleg':
        return getActiveLegStartlist(startListId)

    return (None, None)

1:6: N802 function name '_parseStartlist' should be lowercase
3:6: N806 variable 'startList' in function should be lowercase
4:6: N806 variable 'startListItems' in function should be lowercase
11:6: N802 function name 'getActiveLegStartlist' should be lowercase
11:28: N803 argument name 'startListId' should be lowercase
19:6: N802 function name 'getStartlist' should be lowercase
19:19: N803 argument name 'startListId' should be lowercase


In [111]:
startListId = 451
getStartlist(startListId)

(   startListId  eventId publishedStatus      name
 0          451      124       Published  Thursday,
     startListItemId  startListId  entryId         startDateTime  \
 0             20891          451    20745  2020-01-23T17:37:00Z   
 1             20892          451    20743  2020-01-23T17:36:00Z   
 2             20893          451    20722  2020-01-23T17:35:00Z   
 3             20894          451    20741  2020-01-23T17:34:00Z   
 4             20895          451    20740  2020-01-23T17:33:00Z   
 ..              ...          ...      ...                   ...   
 83            20974          451    20702  2020-01-23T16:55:00Z   
 84            20975          451    20703  2020-01-23T16:54:00Z   
 85            20976          451    20707  2020-01-23T16:53:00Z   
 86            20977          451    20728  2020-01-23T17:15:00Z   
 87            20978          451    20683  2020-01-23T16:00:00Z   
 
            startDateTimeLocal  order  
 0   2020-01-23T18:37:00+01:00     64  

1:2: N816 variable 'startListId' in global scope should not be mixedCase


In [112]:
startList,startListItems = getActiveLegStartlist(startListId)
display(startList.head())
display(startListItems.head())

,startListId,eventId,publishedStatus,name
0,451,124,Published,Thursday


,startListItemId,startListId,entryId,startDateTime,startDateTimeLocal,order
0,20891,451,20745,2020-01-23T17:37:00Z,2020-01-23T18:37:00+01:00,64
1,20892,451,20743,2020-01-23T17:36:00Z,2020-01-23T18:36:00+01:00,63
2,20893,451,20722,2020-01-23T17:35:00Z,2020-01-23T18:35:00+01:00,62
3,20894,451,20741,2020-01-23T17:34:00Z,2020-01-23T18:34:00+01:00,61
4,20895,451,20740,2020-01-23T17:33:00Z,2020-01-23T18:33:00+01:00,60


1:2: N816 variable 'startList' in global scope should not be mixedCase
1:2: N816 variable 'startListItems' in global scope should not be mixedCase
1:10: E231 missing whitespace after ','


In [115]:
def _parseCars(r):
    """Parser for raw cars response."""
    cars = json_normalize(r.json()).drop(columns='eventClasses')
    classes = json_normalize(r.json(), 'eventClasses', meta='entryId')
    return (cars, classes)


def getCars(sdbRallyId, raw=False, func=_parseCars):
    """Get cars for a specified rally."""
    args = {"command": "getCars", "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=2, raw=raw)

1:6: N802 function name '_parseCars' should be lowercase
8:6: N802 function name 'getCars' should be lowercase
8:14: N803 argument name 'sdbRallyId' should be lowercase
10:80: E501 line too long (80 > 79 characters)


In [116]:
cars, classes = getCars(sdbRallyId)
display(cars.head())
display(classes.head())
cars.head().columns

,tag,entryId,eventId,driverId,codriverId,manufacturerId,entrantId,groupId,tagId,entryListOrder,...,codriver.fullName,codriver.code,manufacturer.manufacturerId,manufacturer.name,manufacturer.logoFilename,entrant.entrantId,entrant.name,entrant.logoFilename,group.groupId,group.name
0,None,3442,78,670,3027,13,965,10,None,1,...,Julien INGRASSIA,ING,13,Citroen,citroen,965,CITROEN TOTAL WRT,,10,WRC
1,None,3443,78,762,4883,33,1,10,None,2,...,Nicolas GILSOUL,GIL,33,Hyundai,hyundai,1,HYUNDAI SHELL MOBIS WRT,None,10,WRC
2,None,3444,78,524,525,84,91,10,None,3,...,Martin JÄRVEOJA,JAR,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC
3,None,3445,78,526,1244,84,91,10,None,4,...,Miikka ANTTILA,ANT,84,Toyota,toyota,91,TOYOTA GAZOO RACING WRT,None,10,WRC
4,None,3446,78,548,549,13,965,10,None,5,...,Janne FERM,FER,13,Citroen,citroen,965,CITROEN TOTAL WRT,,10,WRC


,eventClassId,eventId,name,entryId
0,342,78,RC1,3442
1,342,78,RC1,3443
2,342,78,RC1,3444
3,342,78,RC1,3445
4,342,78,RC1,3446


Index(['tag', 'entryId', 'eventId', 'driverId', 'codriverId', 'manufacturerId',
       'entrantId', 'groupId', 'tagId', 'entryListOrder', 'identifier',
       'vehicleModel', 'eligibility', 'priority', 'status', 'tyreManufacturer',
       'driver.personId', 'driver.countryId', 'driver.country.countryId',
       'driver.country.name', 'driver.country.iso2', 'driver.country.iso3',
       'driver.firstName', 'driver.lastName', 'driver.abbvName',
       'driver.fullName', 'driver.code', 'codriver.personId',
       'codriver.countryId', 'codriver.country.countryId',
       'codriver.country.name', 'codriver.country.iso2',
       'codriver.country.iso3', 'codriver.firstName', 'codriver.lastName',
       'codriver.abbvName', 'codriver.fullName', 'codriver.code',
       'manufacturer.manufacturerId', 'manufacturer.name',
       'manufacturer.logoFilename', 'entrant.entrantId', 'entrant.name',
       'entrant.logoFilename', 'group.groupId', 'group.name'],
      dtype='object')

In [123]:
def _parseRally(r):
    """Parser for raw rally response."""
    rally = json_normalize(r.json()).drop(columns=['eligibilities', 'groups'])
    eligibilities = json_normalize(r.json(), 'eligibilities', meta='rallyId')
    eligibilities.rename(columns={0: 'category'}, inplace=True)
    groups = json_normalize(r.json(), 'groups', meta='rallyId')
    return (rally, eligibilities, groups)


def getRally(sdbRallyId, raw=False, func=_parseRally):
    """Get rally details for specified rally."""
    args = {"command": "getRally",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=3, raw=raw,
                                    renamecols={'rallyId': 'externalIdRally',
                                                'eventId': 'externalIdEvent'},
                                    extracols={'sdbRallyId': sdbRallyId})

1:6: N802 function name '_parseRally' should be lowercase
10:6: N802 function name 'getRally' should be lowercase
10:15: N803 argument name 'sdbRallyId' should be lowercase


In [124]:
rally, eligibilities, groups = getRally(sdbRallyId)
display(rally.head())
display(eligibilities.head())
display(groups.head())

,externalIdRally,externalIdEvent,itineraryId,name,isMain,eventClasses,sdbRallyId
0,94,78,144,WRC,True,None,100


,category,externalIdRally,sdbRallyId
0,M,94,100
1,WRC2PRO,94,100
2,WRC2,94,100
3,RGT,94,100
4,None,94,100


,groupId,name,externalIdRally,sdbRallyId
0,10,WRC,94,100
1,8,R5,94,100
2,9,RGT,94,100
3,71,R3C,94,100
4,7,R3T,94,100


In [127]:
def _parseOverall(r):
    """Parser for raw overall response."""
    overall = json_normalize(r.json())
    return overall


def getOverall(sdbRallyId, stageId, raw=False, func=_parseOverall):
    """Get overall standings for specified rally and stage."""
    args = {"command": "getOverall",
            "context": {"sdbRallyId": _jsInt(sdbRallyId),
                        "activeStage": {"stageId": _jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=1,
                                    raw=raw, extracols={'stageId': stageId})

1:6: N802 function name '_parseOverall' should be lowercase
7:6: N802 function name 'getOverall' should be lowercase
7:17: N803 argument name 'sdbRallyId' should be lowercase


In [128]:
stageId = 1528
overall = getOverall(sdbRallyId, stageId)
overall.head()

""


1:2: N816 variable 'stageId' in global scope should not be mixedCase


In [131]:
def _parseSplitTimes(r):
    """Parser for raw splittimes response."""
    splitPoints = json_normalize(r.json(), 'splitPoints')
    entrySplitPointTimes = json_normalize(r.json(),
                                          'entrySplitPointTimes',
                                          meta='stageId').drop(columns='splitPointTimes')
    splitPointTimes = json_normalize(r.json(),
                                     ['entrySplitPointTimes', 'splitPointTimes'],
                                     meta='stageId')
    return (splitPoints, entrySplitPointTimes, splitPointTimes)


def getSplitTimes(sdbRallyId, stageId,
                  raw=False, func=_parseSplitTimes):
    """Get split times for specified rally and stage."""
    args = {"command": "getSplitTimes",
            "context": {"sdbRallyId": _jsInt(sdbRallyId),
                        "activeStage": {"stageId": _jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=3, raw=raw)

1:6: N802 function name '_parseSplitTimes' should be lowercase
3:6: N806 variable 'splitPoints' in function should be lowercase
4:6: N806 variable 'entrySplitPointTimes' in function should be lowercase
6:80: E501 line too long (89 > 79 characters)
7:6: N806 variable 'splitPointTimes' in function should be lowercase
8:80: E501 line too long (81 > 79 characters)
13:6: N802 function name 'getSplitTimes' should be lowercase
13:20: N803 argument name 'sdbRallyId' should be lowercase


In [132]:
splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId, stageId)
display(splitPoints.head())
display(entrySplitPointTimes.head())
display(splitPointTimes.head())

KeyError: "['splitPointTimes'] not found in axis"

1:2: N816 variable 'splitPoints' in global scope should not be mixedCase
1:2: N816 variable 'entrySplitPointTimes' in global scope should not be mixedCase
1:2: N816 variable 'splitPointTimes' in global scope should not be mixedCase
1:78: E231 missing whitespace after ','
1:80: E501 line too long (86 > 79 characters)


In [134]:
def _parseStageTimes(r):
    """Parser for raw stagetimes response."""
    stagetimes = json_normalize(r.json())
    return stagetimes


def getStageTimes(sdbRallyId, stageId, raw=False, func=_parseStageTimes):
    """Get stage times for specified rally and stage"""
    args = {"command": "getStageTimes",
            "context": {"sdbRallyId": _jsInt(sdbRallyId),
                        "activeStage": {"stageId": _jsInt(stageId)}}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

1:6: N802 function name '_parseStageTimes' should be lowercase
6:1: E302 expected 2 blank lines, found 1
6:6: N802 function name 'getStageTimes' should be lowercase
6:20: N803 argument name 'sdbRallyId' should be lowercase
7:1: D400 First line should end with a period


In [135]:
stagetimes = getStageTimes(sdbRallyId, stageId)
stagetimes.head()

""


1:38: E231 missing whitespace after ','


In [136]:
def _parseStagewinners(r):
    """Parser for raw stagewinners response."""
    stagewinners = json_normalize(r.json())
    return stagewinners

 
def getStagewinners(sdbRallyId, raw=False, func=_parseStagewinners):
    """Get stage winners for specified rally."""
    args = {"command": "getStagewinners",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

1:6: N802 function name '_parseStagewinners' should be lowercase
6:1: E302 expected 2 blank lines, found 1
6:6: N802 function name 'getStagewinners' should be lowercase
6:22: N803 argument name 'sdbRallyId' should be lowercase
22: E231 missing whitespace after ':'
22: E231 missing whitespace after ':'
36: E231 missing whitespace after ':'


In [31]:
stagewinners = getStagewinners(sdbRallyId)
stagewinners.head()

,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,1538,20684,Bayons - Bréziers,983700,00:16:23.7000000
1,1528,20685,Malijai - Puimichel (Live TV),593400,00:09:53.4000000
2,1533,20686,Curbans - Venterol 1,802000,00:13:22
3,1534,20686,Saint-Clément - Freissinières 1,703300,00:11:43.3000000
4,1535,20686,Avançon - Notre-Dame-du-Laus 1,780700,00:13:00.7000000


Should we return empty dataframes with appropriate columns, or `None`?

An advantage of returning an empty dataframe with labelled columns is that we can also use the column value list as a test of a returned column.

We need to be consistent so we can have a common, consistent way of dealing with empty responses. This means things like `is None` or `pd.DataFrame().empty` both have to be handled.

In [139]:
# COLS_PENALTIES=['penaltyId','controlId','entryId','penaltyDurationMs','penaltyDuration','reason']


def _parsePenalties(r):
    """Parser for raw penalties response."""
    penalties = json_normalize(r.json())
    return penalties


def getPenalties(sdbRallyId, raw=False, func=_parsePenalties):
    """Get penalties for specified rally."""
    args = {"command": "getPenalties",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

4:6: N802 function name '_parsePenalties' should be lowercase
10:6: N802 function name 'getPenalties' should be lowercase
10:19: N803 argument name 'sdbRallyId' should be lowercase


In [33]:
penalties = getPenalties(sdbRallyId)
penalties.head()

,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE
3,728,6590,20764,50000,PT50S,5 MINS LATE
4,729,6590,20769,10000,PT10S,1 MIN LATE


In [140]:
# COLS_RETIREMENT = ['retirementId','controlId','entryId','reason','retirementDateTime','retirementDateTimeLocal','status']

def _parseRetirements(r):
    """Parser for raw retirements response."""
    retirements = json_normalize(r.json())
    return retirements


def getRetirements(sdbRallyId, raw=False, func=_parseRetirements):
    """Get retirements for specified rally."""
    args = {"command": "getRetirements",
            "context": {"sdbRallyId": _jsInt(sdbRallyId)}}

    return _get_and_handle_response(URL, args, func, nargs=1, raw=raw)

1:1: E265 block comment should start with '# '
1:80: E501 line too long (122 > 79 characters)
3:1: E302 expected 2 blank lines, found 1
3:6: N802 function name '_parseRetirements' should be lowercase
4:47: W291 trailing whitespace
8:1: E302 expected 2 blank lines, found 1
8:6: N802 function name 'getRetirements' should be lowercase
8:21: N803 argument name 'sdbRallyId' should be lowercase
22: E231 missing whitespace after ':'
22: E231 missing whitespace after ':'
36: E231 missing whitespace after ':'


In [35]:
retirements = getRetirements(sdbRallyId)
retirements.head()

,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent
3,1478,6545,20687,REJOINED,2020-01-24T06:16:00Z,0001-01-01T00:00:00+00:00,Rejoined
4,1479,6545,20710,REJOINED,2020-01-24T06:17:00Z,0001-01-01T00:00:00+00:00,Rejoined


In [36]:
SEASON_URL = 'https://www.wrc.com/ajax.php?contelPageId=186641'

In [37]:
#How can we look these up?
SEASON_CATEGORIES = {'WRC':"35", "WRC2":"46", "WRC3":"49", "JWRC":"58"}

In [141]:
def _parseSeasonCategory(r):
    """Parser for raw season category response."""
    season_category = json_normalize(r.json())
    return season_category


def getSeasonCategory(seasonCategory=SEASON_CATEGORIES['WRC'],
                      raw=False, func=_parseSeasonCategory):
    """Get championships in season category."""
    args = {"command": "getSeasonCategory",
            "context": {"seasonCategory": seasonCategory}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=1, raw=raw)


1:6: N802 function name '_parseSeasonCategory' should be lowercase
6:1: E302 expected 2 blank lines, found 1
6:6: N802 function name 'getSeasonCategory' should be lowercase
6:24: N803 argument name 'seasonCategory' should be lowercase
6:80: E501 line too long (101 > 79 characters)
6:102: W291 trailing whitespace
22: E231 missing whitespace after ':'
22: E231 missing whitespace after ':'
40: E231 missing whitespace after ':'


In [39]:
getSeasonCategory()

,id,externalIdDriver,externalIdCoDriver,externalIdManufacturer,season.id,season.year,season.externalId,season.active,category.id,category.name,category.sorting
0,35,37,38,39,19,2020,6,True,7,WRC,1


In [142]:
# TO DO - what about other seasons?

def getSeasonCategories(seasonCategories=None):
    """Create dataframe of external season categoties."""
    champs = pd.DataFrame()

    if seasonCategories is None:
        for sc in SEASON_CATEGORIES:
            seasonCategory = SEASON_CATEGORIES[sc]

            champs = champs.append(getSeasonCategory(seasonCategory))  # [SC_COLS])
    champs.reset_index(inplace=True, drop=True)
    return champs


3:1: E302 expected 2 blank lines, found 1
3:6: N802 function name 'getSeasonCategories' should be lowercase
3:26: N803 argument name 'seasonCategories' should be lowercase
5:11: E225 missing whitespace around operator
9:14: N806 variable 'seasonCategory' in function should be lowercase
11:80: E501 line too long (83 > 79 characters)


In [41]:
getSeasonCategories()

,id,externalIdDriver,externalIdCoDriver,externalIdManufacturer,season.id,season.year,season.externalId,season.active,category.id,category.name,category.sorting
0,35,37,38,39,19,2020,6,True,7,WRC,1
1,46,40,41,43,19,2020,6,True,12,WRC 2,2
2,49,44,45,None,19,2020,6,True,24,WRC 3,3
3,58,46,47,None,19,2020,6,True,23,JWRC,4


In [143]:
SC_COLS = ['id', 'category.name', 'externalIdDriver',
           'externalIdCoDriver', 'externalIdManufacturer']


def getSeasonChampionshipCodes():
    """Get championship codes in an easily retrieved way."""
    champs = getSeasonCategories()[SC_COLS]
    champs.rename(columns={'externalIdDriver': 'drivers',
                           'externalIdCoDriver': 'codrivers',
                           'externalIdManufacturer': 'manufacturers'},
                  inplace=True)
    return champs

4:1: E302 expected 2 blank lines, found 1
4:6: N802 function name 'getSeasonChampionshipCodes' should be lowercase
5:1: D401 First line should be in imperative mood; try rephrasing
5:80: E501 line too long (84 > 79 characters)


In [43]:
getSeasonChampionshipCodes()

,id,category.name,drivers,codrivers,manufacturers
0,35,WRC,37,38,39
1,46,WRC 2,40,41,43
2,49,WRC 3,44,45,None
3,58,JWRC,46,47,None


In [153]:
def _getChampionshipId(category='WRC', typ='drivers'):
    """Look up external ids for championship by category and championship."""
    champs = getSeasonChampionshipCodes()
    championship_activeExternalId = champs.set_index('id').to_dict(orient='index')[int(SEASON_CATEGORIES[category])]
    activeExternalId = championship_activeExternalId[typ]
    return activeExternalId


def _getSeasonId():
    event, days, channels = getActiveRally()
    return int(event.loc[0, 'season.externalId'])


def _parseChampionship(r):
    """Parser for raw championship response."""
    championship = json_normalize(r.json()).drop(columns=['championshipRounds',
                                                          'championshipEntries'])
    championshipRounds = json_normalize(r.json(), 'championshipRounds')
    championshipEntries = json_normalize(r.json(), 'championshipEntries')
    return (championship, championshipRounds, championshipEntries)


def getChampionship(category='WRC', typ='drivers', season_external_id=None,
                    raw=False, func=_parseChampionship):
    """
    Get Championship details for specified category and championship.

    If nor season ID is provided, use the external seasonid from the active rally.
    """
    season_external_id = _getSeasonId()
    args = {"command": "getChampionship",
            "context": {"season": {"externalId": season_external_id},
                        "activeExternalId": _getChampionshipId(category, typ)}}

    return _get_and_handle_response(SEASON_URL, args, func,
                                    nargs=3, raw=raw)

1:6: N802 function name '_getChampionshipId' should be lowercase
4:6: N806 variable 'championship_activeExternalId' in function should be lowercase
4:80: E501 line too long (116 > 79 characters)
5:6: N806 variable 'activeExternalId' in function should be lowercase
9:6: N802 function name '_getSeasonId' should be lowercase
14:6: N802 function name '_parseChampionship' should be lowercase
17:80: E501 line too long (81 > 79 characters)
18:6: N806 variable 'championshipRounds' in function should be lowercase
19:6: N806 variable 'championshipEntries' in function should be lowercase
23:6: N802 function name 'getChampionship' should be lowercase
28:80: E501 line too long (82 > 79 characters)


In [147]:
(championship, championshipRounds, championshipEntries) = getChampionship()
display(championship)
display(championshipRounds.head())
display(championshipEntries.head())

,championshipId,seasonId,name,type,fieldOneDescription,fieldTwoDescription,fieldThreeDescription,fieldFourDescription,fieldFiveDescription
0,37,6,FIA World Rally Championship for Drivers,Drivers,FirstName,LastName,CountryISO3,Manufacturer,TyreManufacturer


,eventId,championshipId,order,event.eventId,event.countryId,event.country.countryId,event.country.name,event.country.iso2,event.country.iso3,event.name,...,event.timeZoneOffset,event.surfaces,event.organiserUrl,event.categories,event.mode,event.trackingEventId,event.clerkOfTheCourse,event.stewards,event.templateFilename,event.locked
0,124,37,1,124,147,147,Monaco,MC,MCO,88e Rallye Automobile Monte-Carlo,...,60,Asphalt,https://acm.mc/en/edition/rallye-monte-carlo-e...,None,Rally,3044,Alain Pallanca,Waltraud Wünsch ...,results-report-templates/8f837fca-d5d3-4049-a5...,False
1,125,37,2,125,215,215,Sweden,SE,SWE,Rally Sweden,...,60,,,None,Rally,3046,None,None,None,False
2,126,37,3,126,144,144,Mexico,MX,MEX,Rally Guanajuato Mexico,...,-360,,,None,Rally,3047,None,None,None,False
3,127,37,4,127,11,11,Argentina,AR,ARG,SpeedAgro Rally Argentina,...,-180,,,None,Rally,3048,None,None,None,False
4,128,37,5,128,178,178,Portugal,PT,PRT,Rally de Portugal,...,0,,,None,Rally,3049,None,None,None,False


,championshipEntryId,championshipId,personId,entrantId,manufacturerId,tyreManufacturer,fieldOne,fieldTwo,fieldThree,fieldFour,fieldFive
0,751,37,762,None,33,Michelin,Thierry,NEUVILLE,BEL,hyundai,Michelin
1,753,37,534,None,84,Michelin,Elfyn,EVANS,GBR,toyota,Michelin
2,757,37,670,None,84,Michelin,Sébastien,OGIER,FRA,toyota,Michelin
3,758,37,548,None,26,Michelin,Esapekka,LAPPI,FIN,ford,Michelin
4,760,37,700,None,84,Michelin,Kalle,ROVANPERÄ,FIN,toyota,Michelin


1:3: N816 variable 'championshipRounds' in global scope should not be mixedCase
1:3: N816 variable 'championshipEntries' in global scope should not be mixedCase


In [46]:
getSeasonCategories().to_dict(orient='index')  # [int(SEASON_CATEGORIES['JWRC'])]

{0: {'id': 35,
  'externalIdDriver': 37,
  'externalIdCoDriver': 38,
  'externalIdManufacturer': 39,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True,
  'category.id': 7,
  'category.name': 'WRC',
  'category.sorting': 1},
 1: {'id': 46,
  'externalIdDriver': 40,
  'externalIdCoDriver': 41,
  'externalIdManufacturer': 43,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True,
  'category.id': 12,
  'category.name': 'WRC 2',
  'category.sorting': 2},
 2: {'id': 49,
  'externalIdDriver': 44,
  'externalIdCoDriver': 45,
  'externalIdManufacturer': None,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True,
  'category.id': 24,
  'category.name': 'WRC 3',
  'category.sorting': 3},
 3: {'id': 58,
  'externalIdDriver': 46,
  'externalIdCoDriver': 47,
  'externalIdManufacturer': None,
  'season.id': 19,
  'season.year': 2020,
  'season.externalId': 6,
  'season.active': True

In [148]:
def _parseChampionshipStandings(r):
    """Parser for raw champioship standings response."""
    championship_standings = json_normalize(r.json(),
                                            'entryResults',
                                            meta='championshipId')
    if not championship_standings.empty:
        championship_standings.drop(columns='roundResults', inplace=True)
        round_results = json_normalize(r.json(),
                                       ['entryResults', 'roundResults'])
    else:
        round_results = pd.DataFrame()
    return (championship_standings, round_results)


def getChampionshipStandings(category='WRC', typ='drivers',
                             season_external_id=None,
                             raw=False, func=_parseChampionshipStandings):
    """Get championship standings."""
    season_external_id = _getSeasonId()
    args = {"command": "getChampionshipStandings",
            "context": {"season": {"externalId": season_external_id},
                        "activeExternalId": _getChampionshipId(category, typ)}}

    return _get_and_handle_response(SEASON_URL, args, func, nargs=2,
                                    raw=raw, extracols={'category': category,
                                                        'championship': typ})

1:6: N802 function name '_parseChampionshipStandings' should be lowercase
15:6: N802 function name 'getChampionshipStandings' should be lowercase
15:44: E231 missing whitespace after ','
16:54: W291 trailing whitespace
21:68: E231 missing whitespace after ','
22:72: E231 missing whitespace after ','
67: E231 missing whitespace after ':'
71: E231 missing whitespace after ':'


In [145]:
championship_standings, round_results = getChampionshipStandings('WRC')
display(championship_standings.head())
display(round_results.head())

,championshipEntryId,overallPosition,overallPoints,championshipId,category,championship
0,751,1,30,37,WRC,drivers
1,757,2,22,37,WRC,drivers
2,753,3,17,37,WRC,drivers
3,758,4,13,37,WRC,drivers
4,760,5,10,37,WRC,drivers


,championshipEntryId,championshipId,eventId,position,totalPoints,pointsBreakdown,dropped,status,publishedStatus,category,championship
0,751,37,124,1,30,25 + 5,False,Finished,Published,WRC,drivers
1,757,37,124,2,22,18 + 4,False,Finished,Published,WRC,drivers
2,753,37,124,3,17,15 + 2,False,Finished,Published,WRC,drivers
3,758,37,124,4,13,12 + 1,False,Finished,Published,WRC,drivers
4,760,37,124,5,10,10,False,Finished,Published,WRC,drivers
